## Import libraries

In [1]:
import folium
import pandas as pd
import json
from numpy import median
from folium import Choropleth, Circle, Marker, Icon, Map
import ast

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/df_api.csv')

In [3]:
df.head()

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,num_vegan_rest,preschool,num_preschool,starbucks,num_starbucks,airport,num_airport,clubs,num_clubs,weighted_punct
0,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98
1,Aviary,$19M,USA,NY,40.749376,-73.996435,243 West 30th Street,11th Floor,10001,"[{'fsq_id': '5f161e6f978e663e91ab7155', 'categ...",10,"[{'fsq_id': '4d370251e4b4a09380322936', 'categ...",2,"[{'fsq_id': '55a555fd498e1727ae82a345', 'categ...",10,"[{'fsq_id': '4adb6119f964a520a72621e3', 'categ...",10,"[{'fsq_id': '4cc0b629f82ebfb7b4af6f91', 'categ...",8,821.40
2,Gilt Groupe,$236M,USA,NY,40.747270,-73.980064,2 Park Ave Fl 4,NaN,10016-5602,"[{'fsq_id': '4b02f734f964a520814b22e3', 'categ...",10,"[{'fsq_id': '52712a6911d25f0e0dfa94cb', 'categ...",1,"[{'fsq_id': '4aa01922f964a520483e20e3', 'categ...",10,"[{'fsq_id': '4a5623d5f964a52081b41fe3', 'categ...",10,"[{'fsq_id': '741d73901e18012f487f54d8', 'categ...",4,732.10
3,99designs,$35M,USA,CA,37.795531,-122.400598,447 Battery St.,3rd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '558a380a498e13330910aef6', 'categ...",1,"[{'fsq_id': '6167c8968b400716f1d65e0b', 'categ...",7,"[{'fsq_id': '535ece06498e564c67f2f049', 'categ...",9,"[{'fsq_id': '4f32bd4d19836c91c7f53884', 'categ...",7,699.97
4,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111,"[{'fsq_id': '4b391baef964a520605625e3', 'categ...",10,"[{'fsq_id': '5cc0a9cac0c0840025deceea', 'categ...",3,"[{'fsq_id': '49d0da91f964a520395b1fe3', 'categ...",3,"[{'fsq_id': 'eb55470d48024203bb09894b', 'categ...",3,"[{'fsq_id': '55ed1abb498e9e55d2e0edfc', 'categ...",9,607.15


In [4]:
df_subset = df[df['weighted_punct'] > 600]

In [5]:
df_subset['vegan_rest'] = df_subset['vegan_rest'].apply(ast.literal_eval)

In [6]:
def pre_explode(x):
    new_list = []
    for i in df_subset['vegan_rest'][0]:
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        new_list.append((lat, lon))
    return new_list

In [7]:
df_subset['vegan_rest_coord'] = df_subset['vegan_rest'].apply(pre_explode)

In [8]:
df_subset = df_subset.explode('vegan_rest_coord')
df_subset = df_subset.reset_index(drop=True)

In [9]:
df_subset = df_subset.drop_duplicates(subset='vegan_rest_coord', keep='first')

In [10]:
df_subset[['vegan_rest_lat', 'vegan_rest_lon']] = df_subset['vegan_rest_coord'].apply(lambda x: pd.Series([x[0], x[1]]))
df_subset.drop('vegan_rest_coord', axis=1, inplace=True)

In [11]:
df_subset

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code,vegan_rest,...,num_preschool,starbucks,num_starbucks,airport,num_airport,clubs,num_clubs,weighted_punct,vegan_rest_lat,vegan_rest_lon
0,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.744946,-73.990249
1,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.743323,-73.994378
2,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.745153,-73.994538
3,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.746151,-73.997547
4,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.745543,-73.994073
5,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.745134,-73.995236
6,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.750153,-73.995215
7,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.747204,-73.996497
8,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.747313,-74.000432
9,Netbiscuits,$27M,USA,NY,40.746754,-73.995323,7th Floor,275 Seventh Ave,10001,"[{'fsq_id': '5287e35711d2ab3ea77f7ebb', 'categ...",...,3,"[{'fsq_id': '506c7556e4b03bdbe5afda50', 'categ...",8,"[{'fsq_id': '4fd2219fe4b03e7cee27e882', 'categ...",10,"[{'fsq_id': '60da36b2d61cae1ce8d45a04', 'categ...",10,824.98,40.742620,-73.996447


In [12]:
df_subset['vegan_rest_2'][0]

KeyError: 'vegan_rest_2'

In [ ]:
ast.literal_eval(df_subset['vegan_rest'][3])

In [ ]:
p1_lat_NY, p1_lon_NY, p2_lat_NY, p2_lon_NY, p3_lat_NY, p3_lon_NY = df['offices_latitude'][0], df['offices_longitude'][0], df['offices_latitude'][1], df['offices_longitude'][1], df['offices_latitude'][2], df['offices_longitude'][2]

In [ ]:
middle_lat_NY = median([p1_lat_NY, p2_lat_NY, p3_lat_NY])
middle_lon_NY = median([p1_lon_NY, p2_lon_NY, p3_lon_NY])

In [ ]:
testmap_NY = folium.Map(location=[middle_lat_NY, middle_lon_NY], zoom_start=13)

In [ ]:
folium.Circle(location=[p1_lat_NY, p1_lon_NY], popup='Point 1A', fill_color='#ff0000', radius=200, weight=2, color="#000000").add_to(testmap_NY)
folium.Circle(location=[p2_lat_NY, p2_lon_NY], popup='Point 1B', fill_color='#00ff00', radius=200, weight=2, color="#000000").add_to(testmap_NY)
folium.Circle(location=[p3_lat_NY, p3_lon_NY], popup='Point 1C', fill_color='#0000ff', radius=200, weight=2, color="#ffffff").add_to(testmap_NY)

In [ ]:
display(testmap_NY)

In [ ]:
for index, row in df.iterrows():
    
    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}
            
    # 1. Check for the conditions of Festividad: to get the diferent Icon
    if row["num_vegan_rest"] != None:
        icon = Icon (
                icon = "laptop",
                prefix = "fa",
                color = "red",
                icon_color = "white")
        
    elif row["num_preschool"] != None:
        icon = Icon (
                icon = "moon-o",
                prefix = "fa",
                color = "white",
                icon_color = "black")
        
    elif row["num_starbucks"] != None:
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
    elif row["num_airport"] != None:
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
        
    elif row["num_clubs"] != None:
        icon = Icon (
                icon = "cutlery",
                prefix = "fa",
                color = "green",
                icon_color = "white")
    
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(testmap_NY)

testmap_NY

In [ ]:
for index, row in df.iterrows():
    
    all_together = {"location": [row["offices_latitude"], row["offices_longitude"]]}
            
    # 1. Check for the conditions of Festividad: to get the diferent Icon
    if row["num_vegan_rest"] != None:
        icon = Icon (
                icon = "laptop",
                prefix = "fa",
                color = "red",
                icon_color = "white")
      
    # 2. With the icon: I pass that to the Marker

    new_marker = Marker(**all_together, icon = icon)
    
    # 3. Add a Marker per row
    
    new_marker.add_to(testmap_NY)

testmap_NY